In [1]:
import dlib

face_detector = dlib.get_frontal_face_detector()
face_landmarks_detector = dlib.shape_predictor("lib/shape_predictor_68_face_landmarks.dat")

In [2]:
import cv2
import glob
import numpy as np

def preProcessImage(image):
  return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def drawFaceLandMarks(face_landmarks, image):
  for n in range(0, 68):
    x = face_landmarks.part(n).x
    y = face_landmarks.part(n).y
    cv2.circle(image, (x, y), 1, (0, 255, 255), 1)

def shape_to_np(shape, dtype="int"):
	coords = np.zeros((68, 2), dtype=dtype)
	for i in range(0, 68):
		coords[i] = (shape.part(i).x, shape.part(i).y)
	return coords

def read_images():
  image_list = []
  for filename in glob.glob('images/*.jpg'):
    image = cv2.imread(filename)
    image_list.append(image)
  return image_list

images = read_images()

for image in images:
  gray = preProcessImage(image)
  faces = face_detector(gray)

  for face in faces:
    face_landmarks = face_landmarks_detector(gray, face)
    drawFaceLandMarks(face_landmarks, image)

  cv2.imshow('face landmarks', image)
  cv2.waitKey(0)
  cv2.destroyWindow('face landmarks')

In [9]:
def get_facelandmarks(image):
    gray = preProcessImage(image)
    face = face_detector(gray)[0]
    face_landmarks = face_landmarks_detector(gray, face)

    return shape_to_np(face_landmarks)


def get_normalized_facelandmarks(image):
    face_landmarks = get_facelandmarks(image)

    return np.subtract(face_landmarks, np.mean(face_landmarks, axis=0))


def affine_register(face1_landmarks, face2_landmarks):
    coefficient_1 = (face2_landmarks[:, 0] * face2_landmarks[:, 0]).sum()
    coefficient_2 = (face2_landmarks[:, 1] * face2_landmarks[:, 0]).sum()
    coefficient_3 = (face2_landmarks[:, 1] * face2_landmarks[:, 1]).sum()

    coefficients_matrix = [[coefficient_1, coefficient_2, 0, 0],
                    [coefficient_2, coefficient_3, 0, 0],
                    [0, 0, coefficient_1, coefficient_2],
                    [0, 0, coefficient_2, coefficient_3]
                    ]

    equation_1_answer = (face1_landmarks[:, 0] * face2_landmarks[:, 0]).sum()
    equation_2_answer = (face1_landmarks[:, 0] * face2_landmarks[:, 1]).sum()
    equation_3_answer = (face2_landmarks[:, 0] * face1_landmarks[:, 1]).sum()
    equation_4_answer = (face1_landmarks[:, 1] * face2_landmarks[:, 1]).sum()

    answers = [equation_1_answer, equation_2_answer,
               equation_3_answer, equation_4_answer]

    result = np.linalg.solve(coefficients_matrix, answers)

    return result.reshape((2,2))

def similarity_register(face1_landmarks, face2_landmarks):
    coefficient = (face2_landmarks[:,0] ** 2 + face2_landmarks[:,1] ** 2).sum()
    equation_1_answer = (face1_landmarks[:,0] * face2_landmarks[:,0] + face1_landmarks[:,1] * face2_landmarks[:,1]).sum()
    equation_2_answer = (face1_landmarks[:,1] * face2_landmarks[:,0] - face1_landmarks[:,0] * face2_landmarks[:,1]).sum()
    coefficients_matrix = [[coefficient, 0], [0, coefficient]]
    answers = [equation_1_answer, equation_2_answer]

    a, b = np.linalg.solve(coefficients_matrix, answers)

    return [[a, -b], [b, a]]

def apply_transform(image, transformMatrix):
    return image @ transformMatrix

def draw_face(image, points, color):
    width = image.shape[0]
    height = image.shape[0]

    for (x, y) in points:
        cv2.circle(image, (int(x + width / 2),
                   int(y + height / 2)), 1, color, 1)


face1_landmarks = get_normalized_facelandmarks(images[0])
face2_landmarks = get_normalized_facelandmarks(images[3])

M = affine_register(face1_landmarks, face2_landmarks)

print(M)
# M = similarity_register(face1_landmarks, face2_landmarks)

registerd_face_2 = apply_transform(face2_landmarks, M)

base = np.zeros((400, 400, 3), np.uint8)

draw_face(base, registerd_face_2, (255, 0, 0))
draw_face(base, face1_landmarks, (0, 0, 255))

cv2.imshow('base', base)
cv2.waitKey(0)
cv2.destroyWindow('base')


[[ 0.64756736  0.06892796]
 [-0.07116891  0.59147101]]
